In [ ]:
# import pandas as pd
# import pandas_ta as ta

# df = pd.DataFrame()
# df.ta.indicators()
# help(ta.rsi)
# help(ta.linear_regression)

In [ ]:
# https://pypi.org/project/isyatirimhisse/
# pip install isyatirimhisse==4.0.0

# pip install pandas_ta
# ModuleNotFoundError: No module named 'pkg_resources'
# pip install --upgrade setuptools

import numpy as np
import pandas as pd
import pandas_ta as ta
from scipy.stats import t
from isyatirimhisse import StockData
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Function to generate buy and sell signals based on RSI and linear regression analysis
def generate_buy_sell_signals(symbol, start_date='01-01-2003', frequency='1d', exchange='0', alpha=0.2, buy_threshold=40, sell_threshold=60, buffer_percentage=0, RSI_length=14):

    stock_data = StockData()
    data = stock_data.get_data(symbols=symbol, start_date=start_date, frequency=frequency, exchange=exchange)

    data = data[['DATE', 'CLOSING_TL']]
    data['CLOSING_TL'] = np.log(data['CLOSING_TL'])
    data = data.rename(columns={'CLOSING_TL': symbol})
    data['DATE'] = pd.to_datetime(data['DATE'])
    data = data.set_index('DATE')

    # Calculating RSI and linear regression using pandas_ta library
    data['RSI'] = ta.rsi(close=data[symbol], length=RSI_length)
    data['Time'] = np.arange(1, len(data) + 1)
    linreg_result = ta.linear_regression(data['Time'], data[symbol])
    data['Fitted'] = linreg_result['line']
    data = data.dropna()

    # Calculating prediction intervals for linear regression
    degrees_of_freedom = len(data) - 2
    t_crit = t.ppf(1 - alpha / 2, degrees_of_freedom)
    beta0 = linreg_result['a']
    beta1 = linreg_result['b']
    data['Lower'] = beta0 + beta1 * data['Time'] - t_crit * np.sqrt(np.mean((data[symbol] - data['Fitted']) ** 2) * (1 + 1 / len(data) + (data['Time'] - data['Time'].mean()) ** 2 / ((data['Time'] - data['Time'].mean()) ** 2).sum()))
    data['Upper'] = beta0 + beta1 * data['Time'] + t_crit * np.sqrt(np.mean((data[symbol] - data['Fitted']) ** 2) * (1 + 1 / len(data) + (data['Time'] - data['Time'].mean()) ** 2 / ((data['Time'] - data['Time'].mean()) ** 2).sum()))

    data['potential_sell'] = (data['RSI'] > sell_threshold) & (data[symbol] > data['Upper'] * (1 + buffer_percentage / 100))
    data['potential_buy'] = (data['RSI'] < buy_threshold) & (data[symbol] < data['Lower'] * (1 - buffer_percentage / 100))

    data['position'] = np.nan
    data['Buy_Signal_Price'] = data.loc[(data['potential_buy'] & ~data['potential_buy'].shift(periods=1, fill_value=False)), symbol]
    data.loc[(data['potential_buy'] & ~data['potential_buy'].shift(periods=1, fill_value=False)), 'position'] = 'buy'
    data['Sell_Signal_Price'] = data.loc[(data['potential_sell'] & ~data['potential_sell'].shift(periods=1, fill_value=False)), symbol]
    data.loc[(data['potential_sell'] & ~data['potential_sell'].shift(periods=1, fill_value=False)), 'position'] = 'sell'

    data.position.iloc[0] = 'sell'
    data['position'] = data.loc[data['position'] != data['position'].fillna(method='ffill').shift(periods=1, fill_value='sell'), 'position']

    # data.to_excel('check_signals.xlsx')

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12, 12))

    ax1.set_title(f'{symbol} - for Educational Purposes Only', fontsize=16)

    ax1.plot(data.index, data[symbol], color='gray', alpha=.5)
    ax1.set_ylabel("Log Price")

    ax1.plot(data.index, data['Fitted'], label='Fitted', linestyle='--', color='blue')
    ax1.plot(data.index, data['Lower'], label='Lower Band', linestyle='--', color='orange')
    ax1.plot(data.index, data['Upper'], label='Upper Band', linestyle='--', color='purple')

    buy_signal_indices = data.index[data['position'] == 'buy']
    ax1.plot(buy_signal_indices, data.loc[buy_signal_indices, symbol], '^', markersize=12, color='green', label='Buy Signal')
    sell_signal_indices = data.index[data['position'] == 'sell']
    ax1.plot(sell_signal_indices, data.loc[sell_signal_indices, symbol], 'v', markersize=12, color='red', label='Sell Signal')

    ax1.legend()

    ax2.plot(data.index, data['RSI'], color='red')
    ax2.axhline(y=buy_threshold, color='black', linestyle='--')
    ax2.axhline(y=sell_threshold, color='black', linestyle='--')

    plt.savefig(f'{symbol}_plot.png')

    plt.show()

    signals_df = pd.DataFrame(columns=['DATE', 'STOCK', 'BUY_SELL', 'PRICE'])
    buy_signals = data.reset_index()[['DATE', 'Buy_Signal_Price']].dropna()
    buy_signals['STOCK'] = symbol
    buy_signals['BUY_SELL'] = 'Buy'
    buy_signals['PRICE'] = np.exp(buy_signals['Buy_Signal_Price'])

    sell_signals = data.reset_index()[['DATE', 'Sell_Signal_Price']].dropna()
    sell_signals['STOCK'] = symbol
    sell_signals['BUY_SELL'] = 'Sell'
    sell_signals['PRICE'] = np.exp(sell_signals['Sell_Signal_Price'])

    if not signals_df.empty:
        signals_df = pd.concat([signals_df, buy_signals[['DATE', 'STOCK', 'BUY_SELL', 'PRICE']]], ignore_index=True)
        signals_df = pd.concat([signals_df, sell_signals[['DATE', 'STOCK', 'BUY_SELL', 'PRICE']]], ignore_index=True)
    else:
        signals_df = pd.concat([buy_signals[['DATE', 'STOCK', 'BUY_SELL', 'PRICE']], sell_signals[['DATE', 'STOCK', 'BUY_SELL', 'PRICE']]], ignore_index=True)

    return signals_df

def main():
    symbols = ['CIMSA','DOAS','SASA']

    all_signals = pd.DataFrame()

    for symbol in symbols:
        signals_df = generate_buy_sell_signals(
            symbol,
            start_date='01-01-2018',
            frequency='1d',
            exchange='0',
            alpha=0.1,
            buy_threshold=40,
            sell_threshold=60,
            buffer_percentage=10,
            RSI_length=50
        )
        all_signals = pd.concat([all_signals, signals_df])

    all_signals.to_excel("all_buy_sell_signals.xlsx", index=False)

if __name__ == "__main__":
    main()